In [1]:
import os
from copy import deepcopy
from tqdm import tqdm
from datetime import date, datetime
import pandas as pd
import numpy as np

In [2]:
df_all = []
file_lst_used = []
file_lst = os.listdir('NMXLNT')
for file_name in file_lst:
    if "VINH NIEM" not in file_name and "HO TAY" not in file_name:
        df = pd.read_csv(f"NMXLNT/{file_name}")
        df = df.drop_duplicates().reset_index().drop("index", axis = 1) # after some inspection, realize there's some duplicates
        df_all.append(df)
        file_lst_used.append(file_name)
        print(file_name)
        print(df.columns)

BAY MAU.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
BTLVT.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
CAU NGA.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
TU SON.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
       'total_p'],
      dtype='object')
YENSO.csv
Index(['datetime', 'flow_in', 'flow_out1', 'flow_out2', 'flow_out3', 'temp',
       'ph', 'tss', 'do', 'cod', 'bod', 'toc', 'no3', 'nh4', 'po4', 'total_n',
 

In [3]:
def preprocessing(df, file_name):
    df["Location"] = file_name
    df["datetime"] = pd.to_datetime(df["datetime"])
    df = df[~df["cod"].isna()].reset_index().drop("index", axis = 1)
    if "flow_in" not in df.columns:
        df["flow_in"] = df["flow_in1"] + df["flow_in2"]
    #print(df["cod"].isna().sum())
    df = df.reset_index().drop("index", axis = 1)
    temp_df = deepcopy(df)
    last_row_with_time_within = {
        j: [] for j in range(4, 9) # store between 4 - 8 hours since we are trying to predict the next 4-8 hours
    }
    curr_row = {
        j: 0 for j in range(4, 9)
    }
    for i in tqdm(range(df.shape[0])):
        for j in range(4, 9):
            # we first try to find until we find first instance that is STRICTLY LESS THAN k hours before current time
            # then the previous instance is the instance that might before around 4 hours before current time
            while curr_row[j] < i and (df.loc[i, "datetime"] - df.loc[curr_row[j], "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j:
                curr_row[j] += 1
            # when we stop is when we do not see a fit row, try to check if that fit row actually fit 
            # a fit would be from k - < k+1 hours before (curr_row - 1 would be our answer)
            if curr_row[j] <= i and curr_row[j] >= 1 and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) >= 60 * j and \
               (df.loc[i, "datetime"] - df.loc[curr_row[j] - 1, "datetime"]) / pd.Timedelta(minutes=1) < 60 * (j+1):
                last_row_with_time_within[j].append(curr_row[j] - 1)
            else:
                # we stop because we have no answer
                last_row_with_time_within[j].append(None)
    for j in range(4, 9):
        df[f"last_row_with_time_within_{j}hour"] = last_row_with_time_within[j]
    # need to separete these two steps since after we inner join, we will lose some rows
    for j in range(4, 9):
        df2 = deepcopy(temp_df.reset_index())
        df = df.merge(df2, how = "inner", left_on = f"last_row_with_time_within_{j}hour", right_on = "index", suffixes=("", f"_prev_{j}"))
        #print(np.mean(((df["datetime"] - df[f"datetime_prev_{j}"]) / pd.Timedelta(minutes=1)) >= 60*j))
    return df

In [4]:
for i in range(len(df_all)):
    df_all[i] = preprocessing(df_all[i], file_lst_used[i])
df_all = pd.concat(df_all, ignore_index = True)
df_all.to_csv("NMXLNT_df.csv", index = False)

100%|██████████| 73179/73179 [01:16<00:00, 961.31it/s] 
